# Primeros pasos con Neo4j

Este notebook es una guía práctica para explorar las capacidades de [Neo4j](https://neo4j.com/), una base de datos de grafos nativa, madura y diseñada para aplicaciones empresariales, tanto transaccionales como analíticas.

Neo4j se destaca por su robustez, su ecosistema completo de herramientas (incluyendo visualización, ciencia de datos y administración) y por su lenguaje de consulta basado en grafos: **Cypher**.

En este notebook aprenderás a:

- Crear una base de datos local con Neo4j Desktop.
- Conectarte desde Python usando el driver oficial.
- Crear nodos y relaciones mediante el lenguaje Cypher.
- Ejecutar consultas para descubrir patrones en el grafo.

Los ejemplos de esta notebook utilizan el lenguaje de consulta **Cypher**.  
Si no estás familiarizado con Cypher, se recomienda revisar el siguiente tutorial oficial de Neo4j para entender su sintaxis y conceptos básicos:[Tutorial de Cypher en Neo4j](https://neo4j.com/docs/getting-started/cypher/)


**Alternativa rápida:**  
Si prefieres evitar la instalación de Neo4j Desktop, también puedes levantar una instancia de Neo4j de forma rápida usando Docker:


```bash
docker run 
  -p 7474:7474\
  -p 7687:7687  \
  -d \
  -e NEO4J_AUTH=neo4j/secretgraph \
  neo4j:latest
```

Esto inicia Neo4j en modo servidor, accesible en [http://localhost:7474](http://localhost:7474) con el usuario `neo4j` y la contraseña que definas.

---

#### 📌 Nota

Este notebook es un **documento vivo**.  
Se irán agregando nuevos ejemplos y casos de uso avanzados de forma progresiva.

¡Siéntete libre de clonar, ejecutar y adaptar estos ejemplos a tus propios proyectos!

---

## Ejemplo 1

In [2]:
# Conectarse a la base de datos 

from neo4j import GraphDatabase

URI = "neo4j://localhost"
AUTH = ("neo4j", "secretgraph")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [4]:
# Crear un grafo de ejemplo  

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    summary = driver.execute_query("""
        CREATE (a:Person {name: $name})
        CREATE (b:Person {name: $friendName})
        CREATE (a)-[:KNOWS]->(b)
        """,
        name="Alice", friendName="David",
        database_="neo4j",
    ).summary
    
    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 2 nodes in 20 ms.


In [7]:
# Consultar un grafo  

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person)-[:KNOWS]->(:Person)
        RETURN p.name AS name
        """,
        database_="neo4j",
    )

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("\nThe query returned {records_count} records in {time} ms.".format(
    records_count=len(records),
    time=summary.result_available_after
))

{'name': 'Alice'}
{'name': 'Alice'}

The query returned 2 records in 1 ms.


## Ejemplo 2

In [8]:
from neo4j import GraphDatabase

# Estas credenciales deben coincidir con las de la base de datos creada en Neo4j
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "secretgraph")

In [9]:
def crear_y_conectar_amigos(driver, nombre_persona, nombre_amigo):
    """
    Crea dos nodos de Persona si no existen y una relación :CONOCE entre ellos.
    Utiliza MERGE para evitar la creación de duplicados.
    """
    
    # MERGE es idempotente: crea el patrón si no existe, o lo encuentra si ya existe.
    # Es la mejor práctica para operaciones de escritura que deben evitar duplicados.
    query = (
        "MERGE (p1:Persona {nombre: $nombre_persona}) "
        "MERGE (p2:Persona {nombre: $nombre_amigo}) "
        "MERGE (p1)-[:CONOCE]->(p2)"
    )
    
    # execute_query es el método recomendado para ejecutar una sola consulta.
    # Los parámetros se pasan como argumentos de palabra clave para prevenir inyecciones de Cypher.
    driver.execute_query(query, nombre_persona=nombre_persona, nombre_amigo=nombre_amigo, database_="neo4j")
    print(f"Relación creada/encontrada entre {nombre_persona} y {nombre_amigo}.")

In [11]:
def encontrar_amigos_de_amigos(driver, nombre_persona):
    """
    Encuentra los 'amigos de amigos' de una persona, excluyendo a la persona original
    y a sus amigos directos.
    """
    
    # Esta consulta de 2 saltos es donde las bases de datos gráficas destacan.
    query = (
        "MATCH (p1:Persona {nombre: $nombre_persona})-[:CONOCE]->(amigo:Persona)-[:CONOCE]->(amigo_de_amigo:Persona) "
        "WHERE p1 <> amigo_de_amigo AND NOT (p1)-[:CONOCE]->(amigo_de_amigo) "
        "RETURN DISTINCT amigo_de_amigo.nombre AS nombre"
    )
    records, summary, keys = driver.execute_query(query, nombre_persona=nombre_persona, database_="neo4j")
    
    # Procesar los resultados
    amigos_de_amigos = [record["nombre"] for record in records]
    return amigos_de_amigos

In [12]:
# El uso de 'with' asegura que el driver se cierre correctamente al finalizar.

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        
        # Verificar la conectividad con el servidor Neo4j
        driver.verify_connectivity()
        print("Conexión exitosa a Neo4j.")
        
        # Poblar el grafo con algunos datos de ejemplo
        crear_y_conectar_amigos(driver, "Alice", "Bob")
        crear_y_conectar_amigos(driver, "Bob", "Charlie")
        crear_y_conectar_amigos(driver, "Alice", "David")
        crear_y_conectar_amigos(driver, "David", "Eve")
        
        # Ejecutar una consulta de lectura
        nombre_a_buscar = "Alice"
        resultados = encontrar_amigos_de_amigos(driver, nombre_a_buscar)
        
        print(f"\nLos amigos de los amigos de {nombre_a_buscar} son: {resultados}")
        
except Exception as e:
    print(f"Se produjo un error al conectar o ejecutar la consulta: {e}")

Conexión exitosa a Neo4j.
Relación creada/encontrada entre Alice y Bob.
Relación creada/encontrada entre Bob y Charlie.
Relación creada/encontrada entre Alice y David.
Relación creada/encontrada entre David y Eve.

Los amigos de los amigos de Alice son: ['Charlie', 'Eve']


## Ejemplo 3

In [15]:
from neo4j import GraphDatabase
import random

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "secretgraph")

In [16]:
# Listas de ejemplo
usuarios = [
    "Lucía", "Mateo", "Sofía", "Lucas", "Martina", "Benjamín", "Emma", "Thiago", "Valentina",
    "Santiago", "Isabella", "Julián", "Camila", "Tomás", "Abril", "Facundo", "Renata", "Juan",
    "Mía", "Agustín", "Catalina", "Dante", "Florencia", "Gaspar", "Lola"
]

peliculas = ["Interestelar", "Matrix", "El Padrino", "Inception", "Avatar"]

# --- Función para insertar usuarios, películas y relaciones ---
def poblar_datos(driver):
    with driver.session(database="neo4j") as session:
        # Crear películas (una sola vez)
        for titulo in peliculas:
            session.run("MERGE (:Pelicula {titulo: $titulo})", titulo=titulo)

        # Crear usuarios y relaciones
        for nombre in usuarios:
            session.run("MERGE (:Usuario {nombre: $nombre})", nombre=nombre)
            peliculas_calificadas = random.sample(peliculas, k=random.randint(2, 5))
            for titulo in peliculas_calificadas:
                rating = random.randint(1, 5)
                session.run("""
                    MATCH (u:Usuario {nombre: $nombre}), (p:Pelicula {titulo: $titulo})
                    MERGE (u)-[:CALIFICO {rating: $rating}]->(p)
                """, nombre=nombre, titulo=titulo, rating=rating)

    print("Base de datos poblada con 25 usuarios, 5 películas y calificaciones aleatorias.")

In [18]:
# Ver calificaciones de un usuario
def mostrar_calificaciones(driver, nombre_usuario):
    query = """
    MATCH (u:Usuario {nombre: $nombre})-[r:CALIFICO]->(p:Pelicula)
    RETURN p.titulo AS pelicula, r.rating AS calificacion
    ORDER BY r.rating DESC
    """
    result, _, _ = driver.execute_query(query, nombre=nombre_usuario, database_="neo4j")
    print(f"Calificaciones de {nombre_usuario}:")
    for row in result:
        print(f"- {row['pelicula']}: {row['calificacion']} estrellas")

In [19]:
# Poniendo en marcha todo
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        poblar_datos(driver)
        mostrar_calificaciones(driver, "Lucía")

except Exception as e:
    print(f"Error al conectar o ejecutar: {e}")

Base de datos poblada con 25 usuarios, 5 películas y calificaciones aleatorias.
Calificaciones de Lucía:
- Avatar: 4 estrellas
- Matrix: 2 estrellas
- Inception: 2 estrellas
- El Padrino: 2 estrellas
